In [2]:
!pip install pandas
!pip install torch

  Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl (12.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 38.7 MB/s eta 0:00:0000:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_10_13_universal2.whl.metadata (4.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached sympy-1.13.3-py3-none-any.w

In [3]:
!pip install matplotlib

  Using cached matplotlib-3.10.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata (11 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.1-cp312-cp312-macosx_10_13_x86_64.whl (8.2 MB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.3 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp312-cp312-macosx_10_13_x86_64.whl (66 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.5 MB/s eta 0:00:00
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)


In [200]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import functools
import re

# Preprocessing

In [ ]:
# Seq processing
trainSeqs = pd.read_csv("stanford-rna-3d-folding/train_sequences.csv")


indexSeqs = trainSeqs["sequence"].apply(list)


nucs = functools.reduce(set.union,indexSeqs.map(set).to_list()) # get all unique nucs at all positions
nucs.add("NONE")

indexSeqs = indexSeqs.apply(pd.Series).fillna("NONE")

nucToIdx = {nuc: i for i, nuc in enumerate(nucs)}


indexSeqs = indexSeqs.map(lambda nuc: nucToIdx[nuc])
indexSeqs.index = trainSeqs["target_id"]
# trainSeqs["sequence"] = trainSeqs["sequence"].
# trainSeqs = pd.get_dummies(trainSeqs, columns=["sequence"])


In [193]:
indexSeqs.head()

,0,1,2,3,4,5,6,7,8,9,...,4288,4289,4290,4291,4292,4293,4294,4295,4296,4297
target_id,,,,,,,,,,,,,,,,,,,,,
1SCL_A,6,6,6,4,6,1,4,1,3,6,...,2,2,2,2,2,2,2,2,2,2
1RNK_A,6,6,1,6,1,3,6,4,6,6,...,2,2,2,2,2,2,2,2,2,2
1RHT_A,6,6,6,3,1,4,6,3,1,6,...,2,2,2,2,2,2,2,2,2,2
1HLX_A,6,6,6,3,4,3,3,1,4,4,...,2,2,2,2,2,2,2,2,2,2
1HMH_E,6,6,1,6,3,1,1,1,4,6,...,2,2,2,2,2,2,2,2,2,2


In [226]:
# label processing

trainLabels = pd.read_csv("stanford-rna-3d-folding/train_labels.csv")
trainLabels["ID"] = trainLabels["ID"].map(lambda x: re.sub("_\d+$","",x))
pivotedLabels = trainLabels.pivot(index="ID",columns="resid",values=["x_1","y_1","z_1"])
pivotedLabels.fillna(0,inplace=True)
pivotedLabels.shape
labelTensor = torch.tensor(pivotedLabels.to_numpy().reshape(pivotedLabels.shape[0],3,-1)).transpose(1,2)
print(labelTensor[0])
pivotedLabels.head(n=1)
# print(indexSeqs.index.unique(),  trainLabels.ID.str.split(r"_(\d+)",expand=True)[0].unique())
# assert indexSeqs.index.unique() == trainLabels.ID.str.split(r"_(\d+)",expand=True)[0].unique()
# trainLabels.head()#pivot(index="ID", columns="resname", values="x_1")

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/var/folders/jd/kcpyjtsx6b7bft_5y_ps8kzm0000gn/T/ipykernel_85215/842584168.py:4: SyntaxWarning: invalid escape sequence '\d'
  trainLabels["ID"] = trainLabels["ID"].map(lambda x: re.sub("_\d+$","",x))


tensor([[ 35.8570, -10.7690,  -7.5480],
        [ 30.2300, -12.0750,  -8.6140],
        [ 23.9680, -11.3560,  -7.6900],
        ...,
        [  0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000]])


x_1                                                    \
resid        1      2       3       4       5       6          7      
ID                                                                    
17RA_A  35.856998  30.23  23.968  19.296  16.362  15.636  16.969999   

                                         ...  z_1                           \
resid        8          9          10    ... 4289 4290 4291 4292 4293 4294   
ID                                       ...                                 
17RA_A  20.391001  24.370001  26.341999  ...  0.0  0.0  0.0  0.0  0.0  0.0   

                            
resid  4295 4296 4297 4298  
ID                          
17RA_A  0.0  0.0  0.0  0.0  

[1 rows x 12894 columns]

In [141]:
class Model(nn.Module):
    def __init__(self, inputDim):
        super().__init__()
        
        self.embedding = nn.Embedding(inputDim, 512)
        encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        self.tBlock1 = nn.TransformerEncoder(encoder_layer, 2)
        self.lastLayer = nn.Linear(512,3)

    def forward(self, input:torch.tensor):
        output = self.embedding(input)
        output = self.tBlock1(output)
        print("no t",output.shape)
        # output = output.transpose(1,2)
        outputShape = output.shape[:2] + (3,)
        output = output.view(-1,512)
        print(output.shape)

        
        output = self.lastLayer(output)
        print(output.shape)

        return output.view(outputShape)

In [227]:
model = Model(len(nucs))
x = torch.tensor(indexSeqs.iloc[:10].values, dtype=torch.int)
print(x)
loss = torch.mean((labelTensor[:10] - model(x)) ** 2)
print(loss)

# # TODO: understand this
# n, d, m = 3, 5, 7
# embedding = nn.Embedding(n, d, max_norm=True)
# W = torch.randn((m, d), requires_grad=True)
# idx = torch.tensor([1, 2])
# a = embedding.weight.clone() @ W.t()  # weight must be cloned for this to be differentiable
# b = embedding(idx) @ W.t()  # modifies weight in-place
# out = (a.unsqueeze(0) + b.unsqueeze(1))
# loss = out.sigmoid().prod()
# loss.backward()

/Users/cazcullimore/dev/rnaFolding/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


tensor([[6, 6, 6,  ..., 2, 2, 2],
        [6, 6, 1,  ..., 2, 2, 2],
        [6, 6, 6,  ..., 2, 2, 2],
        ...,
        [6, 6, 1,  ..., 2, 2, 2],
        [4, 4, 3,  ..., 2, 2, 2],
        [6, 6, 1,  ..., 2, 2, 2]], dtype=torch.int32)
no t torch.Size([10, 4298, 512])
torch.Size([42980, 512])
torch.Size([42980, 3])
tensor(7.0094, grad_fn=<MeanBackward0>)


tensor([[0.7107, 0.3659, 0.8289, 0.5002, 0.8066, 0.2339, 0.7580],
        [0.9072, 0.8802, 0.9501, 0.1265, 0.4863, 0.0402, 0.3243],
        [0.0823, 0.4014, 0.8772, 0.4235, 0.5916, 0.2546, 0.6118],
        [0.9008, 0.4418, 0.9888, 0.8840, 0.0845, 0.5165, 0.0301],
        [0.6144, 0.5111, 0.1143, 0.6712, 0.1091, 0.9249, 0.3197]])


tensor([[0.7107, 0.3659, 0.8289, 0.5002, 0.8066],
        [0.2339, 0.7580, 0.9072, 0.8802, 0.9501],
        [0.1265, 0.4863, 0.0402, 0.3243, 0.0823],
        [0.4014, 0.8772, 0.4235, 0.5916, 0.2546],
        [0.6118, 0.9008, 0.4418, 0.9888, 0.8840],
        [0.0845, 0.5165, 0.0301, 0.6144, 0.5111],
        [0.1143, 0.6712, 0.1091, 0.9249, 0.3197]])